Unfortunately the raw output files are quite large. To make the figures reproducible we created summary statistics using this script. The summaries are available in `data/`.

In [13]:
import numpy as np
import warnings
import json
import os
import pandas as pd

from src_python.cell_tree import CellTree

In [14]:
path = "../data/simulated_data"

n_cells = [100, 100, 50]
n_mut = [50, 100, 100]
n_tests = 100
clones = [5, 10, 20, ""]
stratified = ""  # "_stratified" # stratified means all clones are about equally large

In [15]:
def create_mutation_matrix(parent_vector, mutation_indices, ct):
    n_cells = len(parent_vector)
    n_leaves = int((n_cells+1)/2)
    n_mutations = len(mutation_indices)

    # Initialize mutation matrix with zeros
    mutation_matrix = np.zeros((n_cells, n_mutations), dtype=int)

    # Mark cells with mutations
    for mutation_idx, cell_idx in enumerate(mutation_indices):
        children = [c for c in ct.dfs(cell_idx)]
        for cell in children:  # Traverse all cells below the mutation cell
            mutation_matrix[cell, mutation_idx] = 1  # Mark cells with the mutation

    return mutation_matrix[:n_leaves].T

def create_genotype_matrix(not_selected_genotypes, selected, gt1, gt2, mutation_matrix):
    n_cells = mutation_matrix.shape[1]
    n_loci = len(selected) + len(not_selected_genotypes)
    genotype_matrix = np.full((n_loci, n_cells),"", dtype="str")
    not_selected = [i for i in range(n_loci) if i not in selected]
    # those not selected have a gt independent of the tree learning
    for n, locus in enumerate(not_selected):
        genotype_matrix[locus] = [not_selected_genotypes[n] for _ in range(n_cells)]
    # these are the genes selected for the tree learning
    for n, locus in enumerate(selected):
        genotype_matrix[locus] = np.where(mutation_matrix[n] == 0, gt1[n], np.where(mutation_matrix[n] == 1, gt2[n], mutation_matrix[n]))
    return genotype_matrix


In [ ]:
# summary statistics to compare SCITE-RNA, DENDRO and SClineager on simulated datasets with a variable number of ground truth clones.

genotype_differences = {}
genotype_differences["SCITE-RNA"] = {}

for n_c, n_m in zip(n_cells, n_mut):
    genotype_differences["SCITE-RNA"][f"{n_c}_{n_m}"] = {}
    for clone in clones:
        differences = []
        for t in range(n_tests):
            base_path = os.path.join(path, f"{n_c}c{n_m}m{clone}{stratified}")
            genotype_pred_path = os.path.join(base_path, "sciterna_genotype", f"sciterna_genotype_{t}.txt")
            genotype_path = os.path.join(base_path, "genotype", f"genotype_{t}.txt")
            ref_path = os.path.join(base_path, "ref", f"ref_{t}.txt")
            alt_path = os.path.join(base_path, "alt", f"alt_{t}.txt")
    
            if not os.path.exists(genotype_pred_path):
                os.makedirs(os.path.join(base_path, "sciterna_genotype"), exist_ok=True)
                not_selected_path = os.path.join(base_path, "sciterna_not_selected_genotypes", f"sciterna_not_selected_genotypes_{t}.txt")
                selected_path = os.path.join(base_path, "sciterna_selected_loci", f"sciterna_selected_loci_{t}.txt")
                genotypes_path = os.path.join(base_path, "sciterna_inferred_mut_types", f"sciterna_inferred_mut_types_{t}.txt")
                mut_loc_path = os.path.join(base_path, "sciterna_mut_loc", f"sciterna_mut_loc_{t}.txt")
                parent_path = os.path.join(base_path, "sciterna_parent_vec", f"sciterna_parent_vec_{t}.txt")
                
                if os.path.getsize(not_selected_path) > 0:
                    not_selected_genotypes = np.loadtxt(not_selected_path, dtype=str)
                else:
                    not_selected_genotypes = np.array([])
    
                selected = np.loadtxt(selected_path, dtype=int)
                genotypes = np.loadtxt(genotypes_path, dtype=str)
                mutation_loc = np.loadtxt(mut_loc_path, dtype=int)
                sciterna_parent_vec = np.loadtxt(parent_path, dtype=int)
                gt1 = genotypes[0]
                gt2 = genotypes[1]
                
                ct = CellTree(n_cells)
                ct.use_parent_vec(sciterna_parent_vec)
                ct.mut_loc = mutation_loc
                
                mutation_matrix = create_mutation_matrix(ct.parent_vec, ct.mut_loc, ct)
                genotype = create_genotype_matrix(not_selected_genotypes, selected, gt1, gt2, mutation_matrix)
                np.savetxt(genotype_pred_path, genotype, fmt='%s')
            
            genotype_pred = np.loadtxt(genotype_pred_path, dtype=str)
            gt = np.loadtxt(genotype_path, dtype=str)
            alt = np.loadtxt(alt_path)
            ref = np.loadtxt(ref_path)
            
            mapping_dict = {'A': 1.0, 'H': 0.5, 'R': 0}
            vectorized_map = np.vectorize(lambda x: float(mapping_dict[x]))
            genotype_predicted = vectorized_map(genotype_pred)
            genotype = vectorized_map(gt)
                        
            # difference = np.sum(genotype_predicted != genotype)
            difference = np.sum(np.abs(genotype_predicted - genotype))
            differences.append(difference/(n_c * n_m))

        genotype_differences["SCITE-RNA"][f"{n_c}_{n_m}"][clone] = differences        

genotype_differences["SClineager"] = {}

for n_c, n_m in zip(n_cells, n_mut):
    genotype_differences["SClineager"][f"{n_c}_{n_m}"] = {}
    for clone in clones:
        differences = []
        for t in range(n_tests):
            base_path = os.path.join(path, f"{n_c}c{n_m}m{clone}{stratified}")
            vaf_pred_path = os.path.join(base_path, "sclineager_vaf", f"sclineager_vaf_{t}.txt")
            genotype_path = os.path.join(base_path, "genotype", f"genotype_{t}.txt")
            ref_path = os.path.join(base_path, "ref", f"ref_{t}.txt")
            alt_path = os.path.join(base_path, "alt", f"alt_{t}.txt")
    
            vaf_pred = np.loadtxt(vaf_pred_path, dtype=float)
            gt = np.loadtxt(genotype_path, dtype=str)
            alt = np.loadtxt(alt_path)
            ref = np.loadtxt(ref_path)
            
            mapping_dict = {'A': 1.0, 'H': 0.5, 'R': 0}
            vectorized_map = np.vectorize(lambda x: float(mapping_dict[x]))
            genotype = vectorized_map(gt)
            
            vaf_pred = np.round(vaf_pred * 2) / 2
            difference = np.sum(np.abs(vaf_pred - genotype))
            differences.append(difference/(n_c * n_m))

                
        genotype_differences["SClineager"][f"{n_c}_{n_m}"][clone] = differences        
        
genotype_differences["DENDRO"] = {}

for n_c, n_m in zip(n_cells, n_mut):
    genotype_differences["DENDRO"][f"{n_c}_{n_m}"] = {}
    for clone in clones:
        differences = []
        for t in range(n_tests):
            base_path = os.path.join(path, f"{n_c}c{n_m}m{clone}{stratified}")
            clones_pred_path = os.path.join(base_path, "dendro_clones", f"dendro_clones_{t}.txt")
            clones_pred = np.loadtxt(clones_pred_path, dtype=float)
            genotype_path = os.path.join(base_path, "genotype", f"genotype_{t}.txt")
            ref_path = os.path.join(base_path, "ref", f"ref_{t}.txt")
            alt_path = os.path.join(base_path, "alt", f"alt_{t}.txt")
    
            gt = np.loadtxt(genotype_path, dtype=str)
            alt = np.loadtxt(alt_path)
            ref = np.loadtxt(ref_path)
            with np.errstate(invalid='ignore'):
                vaf_observed = alt/(alt+ref)
            
            mapping_dict = {'A': 1.0, 'H': 0.5, 'R': 0}
            vectorized_map = np.vectorize(lambda x: float(mapping_dict[x]))
            genotype = vectorized_map(gt)
            
            unique_classes = np.unique(clones_pred)
    
            # For each unique class, replace column values with the mean of the columns of that class
            for cls in unique_classes:
                class_indices = np.where(clones_pred == cls)[0]
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore", category=RuntimeWarning)
                    mean_values = np.nanmean(vaf_observed[:, class_indices], axis=1)
                
                # in case the mean is nan replace it with the mean genotype over all cells
                row_nanmean = np.nanmean(vaf_observed, axis=1)
                if np.isnan(row_nanmean).any():
                    raise ValueError("Error: The array contains NaN values.")
                mean_values = np.where(np.isnan(mean_values), row_nanmean, mean_values)
                if np.isnan(mean_values).any():
                    raise ValueError("Error: The array contains NaN values.")
                
                vaf_observed[:, class_indices] = np.tile(mean_values[:, np.newaxis], len(class_indices))
                    
                
            vaf_observed_rounded = np.round(vaf_observed * 2) / 2
            if np.isnan(vaf_observed_rounded).any():
                print("Error: The array contains NaN values.")
            
            difference = np.sum(np.abs(vaf_observed_rounded - genotype))
            differences.append(difference/(n_c * n_m))
                
        genotype_differences["DENDRO"][f"{n_c}_{n_m}"][clone] = differences 
        
with open(r"../data/simulated_data/model_comparison.json", "w") as f:
    json.dump(genotype_differences, f)

In [ ]:
# Summary statistics for multiple myeloma
n_bootstrap = 1000
metastasis_nodes = []
study_num = "mm34"
path = rf"../data/results/{study_num}/bootstrap_1000_snvs_3000_only_ref_to_alt"

for i in range(n_bootstrap):
    path_metastasis_node = os.path.join(path, rf"sciterna_nodes_after_branching\nodes_after_branching_{i}.txt")  
    metastasis_nodes_data = np.loadtxt(path_metastasis_node, dtype=int)
    metastasis_nodes.extend(metastasis_nodes_data)

np.savetxt(os.path.join(path, "metastasis_nodes.txt"), metastasis_nodes, fmt="%d")

# takes a while to run

pairwise_distances = np.zeros((n_bootstrap, n_bootstrap))

mut_indicators = {}
for i in range(n_bootstrap):
    mut_indicators[i] = pd.read_csv(os.path.join(path, "sciterna_mut_indicator", f"sciterna_mut_indicator_{i}.csv"), index_col=0)

for i in range(n_bootstrap):
    print(i)
    for j in range(i, n_bootstrap):
        mi1_filled = mut_indicators[i].fillna(0)
        mi2_filled = mut_indicators[j].fillna(0)

        difference = np.abs(mi1_filled - mi2_filled)

        # Create a mask that marks rows where either df1 or df2 contains NaN values
        valid_rows_mask = ~(mut_indicators[i].isna() | mut_indicators[j].isna()).all(axis=1)

        # Filter out rows with all NaNs in either DataFrame
        valid_difference = difference[valid_rows_mask]
        num_valid_rows = valid_difference.shape[0]
        num_columns = mut_indicators[i].shape[1]
        total_valid_elements = num_valid_rows * num_columns

        distance = valid_difference.sum().sum() / total_valid_elements
        pairwise_distances[i, j] = distance
        pairwise_distances[j, i] = distance

np.savetxt(rf"../data/results/{study_num}/bootstrap_1000_snvs_3000_only_ref_to_alt/pairwise_distances.txt", pairwise_distances, fmt='%f')